In [86]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"
database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [90]:
graph.query('''
MATCH (n:__Entity__)
WHERE n.description IS NOT NULL and apoc.meta.cypher.type(n.description) <> 'STRING'
SET n.description = apoc.text.join(n.description, ", ")
RETURN n.description
''')

[]

In [22]:
graph.query(
    """
MATCH (n:`__Community__`)<-[:IN_COMMUNITY]-()<-[:HAS_ENTITY]-(c)
WITH n, count(distinct c) AS chunkCount
SET n.weight = chunkCount"""
)

[]

In [23]:
from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint='https://lang-chain-dev.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
    azure_deployment='text-embedding-ada-002',
    openai_api_version='2023-05-15'
)

In [7]:
from langchain_community.vectorstores import Neo4jVector
# ! pip3 install -U langchain-huggingface
import os
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/storage/models/embedding_models'
from langchain_huggingface import HuggingFaceEmbeddings
# Choose from https://huggingface.co/spaces/mteb/leaderboard

# embedding = HuggingFaceEmbeddings(model_name="lier007/xiaobu-embedding-v2")

model_path = os.path.join(os.environ['SENTENCE_TRANSFORMERS_HOME'], 'models--lier007--xiaobu-embedding-v2/snapshots/ee0b4ecdf5eb449e8240f2e3de2e10eeae877691')
embedding = HuggingFaceEmbeddings(model_name=model_path)

/storage/pyenv/versions/3.12.5/envs/ACP_LLM_312/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/storage/pyenv/versions/3.12.5/envs/ACP_LLM_312/lib/python3.12/site-packages/sentence_transformers/models/Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded v

In [56]:
from langchain_community.vectorstores import Neo4jVector

lc_retrieval_query = """
WITH collect(node) as nodes
// Entity - Text Unit Mapping
WITH
collect {
    UNWIND nodes as n
    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)
    WITH c, count(distinct n) as freq
    RETURN c.content AS chunkText
    ORDER BY freq DESC
    LIMIT $topChunks
} AS text_mapping,
// Entity - Report Mapping
collect {
    UNWIND nodes as n
    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)
    WHERE c.summary is not null
    WITH c, c.rank as rank, c.weight AS weight
    RETURN c.summary 
    ORDER BY rank, weight DESC
    LIMIT $topCommunities
} AS report_mapping,
// Outside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE NOT m IN nodes and r.description is not null
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topOutsideRels
} as outsideRels,
// Inside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE m IN nodes and r.description is not null
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topInsideRels
} as insideRels,
// Entities description
collect {
    UNWIND nodes as n
    match (n)
    WHERE n.description is not null
    RETURN n.description AS descriptionText
} as entities
// We don't have covariates or claims here
RETURN {Chunks: text_mapping, Reports: report_mapping, 
       Relationships: outsideRels + insideRels, 
       Entities: entities} AS text, 1.0 AS score, {} AS metadata
"""

vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="description_embedding",
                                    node_label='__Entity__', 
                                    embedding_node_property='description_embedding', 
                                    text_node_properties=['description'],
                                    retrieval_query=lc_retrieval_query)

In [57]:
topChunks = 3
topCommunities = 3
topOutsideRels = 10
topInsideRels = 10
topEntities = 10

In [58]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

In [59]:
docs = vectorstore.similarity_search(
    "個人保險首續期繳費怎繳費",
    k=topEntities,
    params={
        "topChunks": topChunks,
        "topCommunities": topCommunities,
        "topOutsideRels": topOutsideRels,
        "topInsideRels": topInsideRels,
    },
)
print(docs[0].page_content)

Entities:
- 首期保險費是保戶首次繳交的保險費用，採用金融機構轉帳作業。
- 因核印失敗、請款失敗或主動異動繳費管道者，其生效日認定依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理。
- 依各險種投保規則辦理
- 轉帳扣款作業需經核印成功且同意承保後始得進行。
- 保戶指的是與保險公司簽約的個人或團體。
- 需填寫授權書，繳款人為非保單關係人時須檢附關係證明文件。相關規定請參照繳費方式及受理應檢附文件。
- 不得為空白授權，應依是否取得保單號碼填寫相關欄位，未填寫完整須重新檢附授權書
- 核印和扣款時間需參照各金融機構的相關規定。
- 需至各銀行臨櫃留存印鑑，核印成功後便能授權扣款。
- 須留存金融機構之印鑑或簽章樣式
Reports:
- 在同一社區中，有兩個重要的節點和一個關鍵的關係。節點包括：

1. 一個沒有特定類型的節點，描述的是「新契約首期保險費繳費管道異動之繳費日認定原則」。
2. 一個類型為「概念」的節點，描述的是「新契約」。具體內容為：因核印失敗、請款失敗或主動異動繳費管道者，其生效日認定依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理。

這兩個節點之間存在一個關係：「新契約」依照「新契約首期保險費繳費管道異動之繳費日認定原則」來進行生效日認定。
- 在同一社區中，有兩個重要的節點和一個關鍵的關係。節點包括：

1. 一個沒有特定類型的節點，描述的是「新契約首期保險費繳費管道異動之繳費日認定原則」。
2. 一個類型為「概念」的節點，描述的是「新契約」。具體內容為：因核印失敗、請款失敗或主動異動繳費管道者，其生效日認定依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理。

這兩個節點之間存在一個關係：「新契約」依照「新契約首期保險費繳費管道異動之繳費日認定原則」來進行生效日認定。
- 在同一個社區中，有以下的資訊：

1. 要保人（Person）：在授權書要保人欄位內須簽章，並且簽章需與要保書相同。
2. 保險費暨保險單借款利息自動轉帳付款授權書（Document）：需要填寫授權書，當繳款人非保單關係人時，需附上關係證明文件，詳情請參照繳費方式及應檢附文件。
3. 授權人（Person）：需留存金融機構的印鑑或簽章樣式。

關係如下：
- 保險費暨保險單借款利息自動轉帳付款授權書需要授權人，授權人需留存金融機構的印鑑或簽章

In [30]:
docs

[Document(page_content='Entities:\n- 首期保險費是保戶首次繳交的保險費用，採用金融機構轉帳作業。\n- 因核印失敗、請款失敗或主動異動繳費管道者，其生效日認定依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理。\n- 依各險種投保規則辦理\n- 轉帳扣款作業需經核印成功且同意承保後始得進行。\n- 保戶指的是與保險公司簽約的個人或團體。\n- 需填寫授權書，繳款人為非保單關係人時須檢附關係證明文件。相關規定請參照繳費方式及受理應檢附文件。\n- 不得為空白授權，應依是否取得保單號碼填寫相關欄位，未填寫完整須重新檢附授權書\n- 核印和扣款時間需參照各金融機構的相關規定。\n- 需至各銀行臨櫃留存印鑑，核印成功後便能授權扣款。\n- 須留存金融機構之印鑑或簽章樣式\nReports:\n- 在同一社區中，有兩個重要的節點和一個關鍵的關係。節點包括：\n\n1. 一個沒有特定類型的節點，描述的是「新契約首期保險費繳費管道異動之繳費日認定原則」。\n2. 一個類型為「概念」的節點，描述的是「新契約」。具體內容為：因核印失敗、請款失敗或主動異動繳費管道者，其生效日認定依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理。\n\n這兩個節點之間存在一個關係：「新契約」依照「新契約首期保險費繳費管道異動之繳費日認定原則」來進行生效日認定。\n- None\n- 在同一個社區中，有以下的資訊：\n\n1. 要保人（Person）：在授權書要保人欄位內須簽章，並且簽章需與要保書相同。\n2. 保險費暨保險單借款利息自動轉帳付款授權書（Document）：需要填寫授權書，當繳款人非保單關係人時，需附上關係證明文件，詳情請參照繳費方式及應檢附文件。\n3. 授權人（Person）：需留存金融機構的印鑑或簽章樣式。\n\n關係如下：\n- 保險費暨保險單借款利息自動轉帳付款授權書需要授權人，授權人需留存金融機構的印鑑或簽章樣式。\n- 保險費暨保險單借款利息自動轉帳付款授權書需要要保人，要保人需在授權書要保人欄位簽章，且簽章需與要保書相同。\nChunks:\n- ，經核印成功 (且 同意承保 )始得進行首期保險費 轉帳扣款作業 。核印、扣款時間 請參照各金融機構核印、 扣款時間及相關規定 。  5. 數位帳戶 請至各銀行臨櫃留存印鑑、核印成功便